#### Author: Arun Ramesh, University of Houston. https://rpglab.github.io/people/Arun-Venkatesh-Ramesh/
#### Source webpage: https://rpglab.github.io/resources/FL-ML-R-SCUC_Python/
#### If you use any codes/data here for your work, please cite the following paper: 
#####       Arun Venkatesh Ramesh and Xingpeng Li, “Feasibility Layer Aided Machine Learning Approach for Day-Ahead Operations”, IEEE Transactions on Power Systems, Apr. 2023.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
np.random.seed(1)

import sys
nums = np.arange(2000)
np.set_printoptions(threshold=sys.maxsize)

import csv
import time
from timeit import default_timer as timer

In [2]:
##### data set 24 Bus 24 Prd

dfX_24 = pd.read_csv("demand24Bus24Prd.txt")
dfY_24 = pd.read_csv("commitment24Bus24Prd.txt")

#dfX_24 = pd.read_csv("demand24BusWBCorr24Prd.txt")
#dfY_24 = pd.read_csv("commitment24BusWBCorr24Prd.txt")

#dfX_24 = pd.read_csv("demand73BusWBCorr24Prd.txt")
#dfY_24 = pd.read_csv("commitment73BusWBCorr24Prd.txt")

#dfX_24 = pd.read_csv("demand118BusWBCorr24Prd.txt")
#dfY_24 = pd.read_csv("commitment118BusWBCorr24Prd.txt")

#dfX_24 = pd.read_csv("demand500BusWBCorr24Prd.txt")
#dfY_24 = pd.read_csv("commitment500BusWBCorr24Prd.txt")

#dfX_24 = pd.read_csv("demandPolishBusWBCorr24Prd_merged.txt")
#dfY_24 = pd.read_csv("commitmentPolishBusWBCorr24Prd_merged.txt")

print(dfX_24.info)
print(dfY_24.info)

x = dfX_24.to_numpy()

# normalize X with Base MVA
x = x/100
#print(x[1])

y = dfY_24.to_numpy()
#print(y[1])

# Shuffle (x, y) in unison as the later parts of x will almost all be larger digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 20% for validation data that we never train over.
split_at = len(x) - len(x) // 5
(x_train, x_test) = x[:split_at], x[split_at:]
(y_train, y_test) = y[:split_at], y[split_at:]


print(len(x_train))
print(len(y_train))

print(len(x_test))
print(len(y_test))

print(indices)

#print(x_train[1])
#print(y_train[1])

#print(x_test[1])
#print(y_test[1])

print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

<bound method DataFrame.info of       90.13728903773256  80.95663922833387  75.11440753144379  \
0             67.493145          60.618843          56.244287   
1            107.022171          96.121765          89.185142   
2            108.926408          97.832052          90.772007   
3             62.949194          56.537702          52.457662   
4             95.801737          86.044153          79.834781   
...                 ...                ...                ...   
1494         101.205071          83.534633          79.818072   
1495          84.521178          78.343324          74.534879   
1496          92.821142          84.133780          89.090335   
1497          78.007085          86.473328          79.682256   
1498          85.612332          75.013867          67.089895   

      128.52909733158162  59.25692149702789  123.5214701628187  \
0              96.240225          44.370493          92.490606   
1             152.605688          70.357168         146

In [71]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=450)
neigh.fit(x_train, y_train)

y_hat_tr_knn = neigh.predict(x_train)
y_hat_ts_knn = neigh.predict(x_test)

print (neigh.score(x_train, y_train))
print (neigh.score(x_test, y_test))

print("train accuracy: {} %".format(100 - np.mean(np.abs(y_train - y_hat_tr_knn)) * 100))
print("test accuracy: {} %".format(100 - np.mean(np.abs(y_test - y_hat_ts_knn)) * 100))

# print(y_hat_tr.shape)
# print(y_hat_tr)
# y_hat_ts = neigh.predict(x_test)
# print(y_hat_ts.shape)
# print(y_hat_ts)



0.0
0.0
train accuracy: 96.1902046579466 %
test accuracy: 96.31002331002331 %


In [107]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
#from sklearn.metrics import plot_confusion_matrix

#plot_confusion_matrix(neigh, x_test, y_test)  
#plt.show()
print (y_test.shape)
print (y_hat_ts.shape)
NgxNt = y_test.shape[1]
M = y_test.shape[0]
#print(confusion_matrix(y_test[:,0], y_hat_ts[:,1]))
print (y_test[:,0].shape)
TN = 0
FP = 0
FN = 0
TP = 0
for i in range(M):
    tn, fp, fn, tp = confusion_matrix(y_test[i,:], y_hat_ts[i,:]).ravel()
    TN = TN + tn
    FP = FP + fp
    FN = FN + fn
    TP = TP + tp

print (TP/(NgxNt*M),TN/(NgxNt*M),FP/(NgxNt*M),FN/(NgxNt*M))

(299, 792)
(299, 792)
(299,)
0.5076602141819533 0.46652984696462957 0.016722408026755852 0.009087530826661262


In [6]:
A_hat_ts = neigh.predict_proba(x_test)
#print(A_hat_ts.shape)
print(A_hat_ts)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [72]:
from sklearn.ensemble import RandomForestClassifier
RFclf = RandomForestClassifier(n_estimators = 1, max_depth=2, random_state=0)
RFclf.fit(x_train, y_train)
print (RFclf.score(x_train, y_train))
print (RFclf.score(x_test, y_test))

y_hat_tr_rf = RFclf.predict(x_train)
y_hat_ts_rf = RFclf.predict(x_test)

print("train accuracy: {} %".format(100 - np.mean(np.abs(y_train - y_hat_tr_rf)) * 100))
print("test accuracy: {} %".format(100 - np.mean(np.abs(y_test - y_hat_ts_rf)) * 100))

0.0
0.0
train accuracy: 96.26526400719949 %
test accuracy: 96.25524475524476 %


In [109]:
NgxNt = y_test.shape[1]
M = y_test.shape[0]
#print(confusion_matrix(y_test[:,0], y_hat_ts[:,1]))
print (y_test[:,0].shape)
TN = 0
FP = 0
FN = 0
TP = 0
for i in range(M):
    tn, fp, fn, tp = confusion_matrix(y_test[i,:], y_hat_ts[i,:]).ravel()
    TN = TN + tn
    FP = FP + fp
    FN = FN + fn
    TP = TP + tp

print (TP/(NgxNt*M),TN/(NgxNt*M),FP/(NgxNt*M),FN/(NgxNt*M))

print ((TP+TN)/(TP+TN+FP+FN))

(299,)
0.5023690078037905 0.4704359650011824 0.012816289990203034 0.014378737204824162
0.9728049728049728


In [14]:
print (y_test[3])
print (y_hat_ts[3])

[0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1
 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1
 1 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1
 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 1 1 1 1 

In [8]:
from sklearn.neural_network import MLPClassifier

nfeat = x_train.shape[1]

MlpClf = MLPClassifier(solver='adam', learning_rate='adaptive', alpha=1e-5, hidden_layer_sizes=(nfeat,), activation='relu', random_state=1, max_iter=75, shuffle=False)
MlpClf.fit(x_train,y_train)

print (MlpClf.score(x_train, y_train))
print (MlpClf.score(x_test, y_test))

y_hat_tr_mlp = MlpClf.predict(x_train)
y_hat_ts_mlp = MlpClf.predict(x_test)

print("train accuracy: {} %".format(100 - np.mean(np.abs(y_train - y_hat_tr_mlp)) * 100))
print("test accuracy: {} %".format(100 - np.mean(np.abs(y_test - y_hat_ts_mlp)) * 100))

0.3408333333333333
0.31438127090301005
train accuracy: 97.4765361952862 %
test accuracy: 97.45658930441539 %


C:\tools\Anaconda3\envs\GPU\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (75) reached and the optimization hasn't converged yet.
  warnings.warn(


In [9]:
NgxNt = y_test.shape[1]
M = y_test.shape[0]

TN = 0
FP = 0
FN = 0
TP = 0
for i in range(M):
    tn, fp, fn, tp = confusion_matrix(y_test[i,:], y_hat_ts_mlp[i,:]).ravel()
    TN = TN + tn
    FP = FP + fp
    FN = FN + fn
    TP = TP + tp

print (TP/(NgxNt*M),TN/(NgxNt*M),FP/(NgxNt*M),FN/(NgxNt*M))

0.5029390898956116 0.4716268031485423 0.011625451842843148 0.01380865511300294


In [21]:
print (y_test[3])
print (y_hat_ts_mlp[3])

[0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1
 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1
 1 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1
 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 1 1 1 1 

In [23]:
print (MlpClf.predict_proba(x_test))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
from sklearn.linear_model import LogisticRegression

TrainScore = []
TrainHardScore = []
TestScore = []
TestHardScore = []
NgxNt = y_train.shape[1]
m_Tr = x_train.shape[0]
m_Ts = x_test.shape[0]

Y_hat_tr_skln_pred = np.zeros((m_Tr,NgxNt))
Y_hat_ts_skln_pred = np.zeros((m_Ts,NgxNt))

Plr = np.zeros((m_Ts,NgxNt))

start = timer()
for i in range(NgxNt):
    #print ("Ng*Nt: ",i)   
    
    if set([0,1]).issubset(set(y_train[:,i])):      
        
        logreg = LogisticRegression(random_state=0, solver='liblinear', max_iter = 100)
        logreg.fit(x_train,y_train[:,i]) #original shape before transposed
        Y_hat_tr_skln_pred[:,i] = logreg.predict(x_train)
        TrainScore.append(logreg.score(x_train,y_train[:,i]))
        Y_hat_ts_skln_pred[:,i] = logreg.predict(x_test)
        TestScore.append(logreg.score(x_test,y_test[:,i]))
        Ptemp = logreg.predict_proba(x_test)
        Plr[:,i] = Ptemp[:,1]
    
    else:
        
        if set([0]).issubset(set(y_train[:,i])):
            Y_hat_tr_skln_pred[:,i] = np.zeros(m_Tr)
        else:
            Y_hat_tr_skln_pred[:,i] = np.ones(m_Tr)
        
        if set([0]).issubset(set(y_test[:,i])):
            Y_hat_ts_skln_pred[:,i] = np.zeros(m_Ts)
            Plr[:,i] = np.zeros(m_Ts)
        else:
            Y_hat_ts_skln_pred[:,i] = np.ones(m_Ts)
            Plr[:,i] = np.ones(m_Ts)
            
        TrainScore.append(1)
        TestScore.append(1)
end = timer()
print("training time: ",end - start)
print ("-------------------------------------------------------")

print("training Score: ", np.mean(TrainScore))
print("testing Score: ", np.mean(TestScore))

print("train accuracy: {} %".format(100 - np.mean(np.abs(y_train - Y_hat_tr_skln_pred)) * 100))
print("test accuracy: {} %".format(100 - np.mean(np.abs(y_test - Y_hat_ts_skln_pred)) * 100))

training time:  39.99096119999999
-------------------------------------------------------
training Score:  0.9795717592592592
testing Score:  0.9755413668457147
train accuracy: 97.95717592592592 %
test accuracy: 97.55202526941657 %


In [5]:
from sklearn.metrics import confusion_matrix
M = y_test.shape[0]

TN = 0
FP = 0
FN = 0
TP = 0
for i in range(M):
    tn, fp, fn, tp = confusion_matrix(y_test[i,:], Y_hat_ts_skln_pred[i,:]).ravel()
    TN = TN + tn
    FP = FP + fp
    FN = FN + fn
    TP = TP + tp

print (TP/(NgxNt*M),TN/(NgxNt*M),FP/(NgxNt*M),FN/(NgxNt*M))

0.5047295699469613 0.4707906827472045 0.01246157224418094 0.012018175061653322


In [6]:
correctSeq = 0
wrongseq = 0

yDiff = y_test - Y_hat_ts_skln_pred

for i in range(299):
    if -1 < np.sum(yDiff[i]) < 1: 
        correctSeq = correctSeq + 1
    else:
        wrongseq = wrongseq + 1
        
print (correctSeq/299)

0.27424749163879597


In [83]:
A = [1, 2, 3]
B = [1,2,4]
if A==B:
    print("yes")
else: 
    print("no")

no


In [89]:
print (y_test[2])

[0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1
 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1
 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 1 0
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1
 1 0 0 1 1 1 0 1 0 0 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1
 1 1 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 1 1 1 1 

In [99]:
print (yDiff[7])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

In [61]:
x_test_round = np.around(x_test, decimals=4)

with open("demand500Bus24PrdCompareClassificationMethodsForAmpl.csv","w+",newline="") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(x_test_round)

In [62]:
with open("commitment_LR_500Bus24PrdTestCompareClassificationMethodsForAmpl.csv","w+",newline="") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(Y_hat_ts_skln_pred)

In [63]:
with open("commitment_MLP_500Bus24PrdTestCompareClassificationMethodsForAmpl.csv","w+",newline="") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(y_hat_ts_mlp)

In [73]:
#same variable for KNN and RF. So re-run the classifier and print to correct file
with open("commitment_KNN_73Bus24PrdTestCompareClassificationMethodsForAmpl.csv","w+",newline="") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(y_hat_ts_knn)

In [74]:
with open("commitment_RF_73Bus24PrdTestCompareClassificationMethodsForAmpl.csv","w+",newline="") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(y_hat_ts_rf)

In [110]:
print (0.5023690078037905+0.4704359650011824+0.012816289990203034+0.014378737204824162)

1.0


In [11]:
Pmlp = MlpClf.predict_proba(x_test)
print (Pmlp.shape)

(299, 792)


In [12]:
A_prediction_test_round = np.around(Pmlp, decimals=6)
#A_prediction_test_round = A_prediction_test_round.T
with open("probabilities_MLP_24Bus24PrdTestSample4Ampl.csv","w+",newline="") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(A_prediction_test_round)

In [68]:
print (Plr.shape)
print (Plr[1])
print (y_test[1])

(319, 2160)
[4.07697507e-03 9.86457740e-01 9.45126526e-01 1.40762816e-01
 8.64553363e-04 1.71775727e-03 0.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 9.97436440e-01
 9.13529854e-01 1.00000000e+00 1.00000000e+00 1.00000000e+00
 3.08823959e-02 7.42888500e-02 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 8.74535327e-04 8.68846112e-04
 0.00000000e+00 2.72157723e-03 0.00000000e+00 1.16654675e-02
 0.00000000e+00 3.54208723e-03 2.68216676e-03 9.26785697e-04
 1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.66093367e-03 3.55291700e-03
 7.33719668e-03 4.41952273e-03 6.23642934e-03 2.66890191e-02
 4.34539005e-03 1.91244424e-03 0.00000000e+00 2.72148718e-03
 3.57098162e

In [7]:
A_prediction_test_round = np.around(Plr, decimals=6)
#A_prediction_test_round = A_prediction_test_round.T
with open("probabilities_LR_24Bus24PrdTestSample4Ampl.csv","w+",newline="") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(A_prediction_test_round)